In [195]:
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import mean
import os
import time

In [141]:
x.rdd.first()[0] ## or x.collect()[0][0]

956.439024390244

In [167]:
st=time.time()
directory_path = '/home/hadoop/sokulee/'
lable = 0
f = open('test.txt', 'w')
for n in range(1,100):
    name = 'A0' + str(n)
    directory_name = directory_path + name

    if os.path.exists(directory_name):
        df_sleep_temp = spark.read.json(directory_name + '/*_sleep.json')
        df_steps_temp = spark.read.json(directory_name + '/*_steps.json')
        df_heart_temp = spark.read.json(directory_name + '/*_heart.json')

        a = df_sleep_temp.select(mean(df_sleep_temp['summary']['totalTimeInBed']).alias('sleep'))
        b = df_steps_temp.select(mean(df_steps_temp['activities-steps'][0]['value']).alias('steps'))
        c = df_heart_temp.select(mean(df_heart_temp['activities-heart'][0]['value']['heartRateZones'][0]['minutes'] 
                    + df_heart_temp['activities-heart'][0]['value']['heartRateZones'][1]['minutes'] 
                    + df_heart_temp['activities-heart'][0]['value']['heartRateZones'][2]['minutes'] 
                    + df_heart_temp['activities-heart'][0]['value']['heartRateZones'][3]['minutes']).alias('time'))
        
        f.write(str(lable) + ' 1:' + str(a.collect()[0][0]) + ' 2:' + str(b.collect()[0][0]) + ' 3:' + str(c.collect()[0][0]) + '\n')
        lable += 1
        
f.close()
print(time.time()-st, 'sec')

191.97672581672668 sec


In [196]:
# Loads data.
dataset = spark.read.format("libsvm")\
                .load("/home/hadoop/test.txt")
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(3,[0,1,2],[245.3...|
|  1.0|(3,[0,1,2],[206.3...|
|  2.0|(3,[0,1,2],[396.4...|
|  3.0|(3,[0,1,2],[358.2...|
|  4.0|(3,[0,1,2],[64.63...|
|  5.0|(3,[0,1,2],[400.3...|
|  6.0|(3,[0,1,2],[435.7...|
|  7.0|(3,[0,1,2],[411.1...|
|  8.0|(3,[0,1,2],[407.7...|
|  9.0|(3,[0,1,2],[136.2...|
| 10.0|(3,[0,1,2],[414.0...|
| 11.0|(3,[0,1,2],[402.2...|
| 12.0|(3,[0,1,2],[401.7...|
| 13.0|(3,[0,1,2],[315.2...|
| 14.0|(3,[0,1,2],[433.3...|
| 15.0|(3,[0,1,2],[466.3...|
| 16.0|(3,[0,1,2],[233.7...|
| 17.0|(3,[0,1,2],[294.7...|
| 18.0|(3,[0,1,2],[372.7...|
| 19.0|(3,[0,1,2],[497.5...|
+-----+--------------------+
only showing top 20 rows



In [219]:
# Trains a k-means model.
kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(dataset)

In [220]:
# Evaluate clustering by computing 
# Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 53472367.7325698


In [221]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[  300.524994    5962.90686675  1100.88254458]
[   353.89273469  15007.116        1245.336     ]
[   313.15693878  11324.62250729   1171.90598335]
[   350.70979592  20852.98         1291.73346939]
[  358.37147474  8553.71578947  1217.25748727]


In [190]:
from pyspark.ml.clustering import BisectingKMeans

# Loads data.
dataset = spark.read.format("libsvm").load("test.txt")

# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(5).setSeed(0)
model = bkm.fit(dataset)

# Evaluate clustering.
cost = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(cost))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)


Within Set Sum of Squared Errors = 92602536.59748363
Cluster Centers: 
[  300.524994    5962.90686675  1100.88254458]
[  346.7764723   8244.93285714  1209.78056669]
[  333.78123057  9879.48        1179.40340834]
[   347.80933673  12742.11719388   1215.33962549]
[   345.10393321  17330.29636364   1264.85640074]


In [191]:
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel

# Load and parse the data
data = sc.textFile("spark-2.1.1-bin-hadoop2.7/data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

Within Set Sum of Squared Error = 0.6928203230275497
